In [82]:
import pandas as pd
import numpy as np

In [83]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [84]:
df = df.iloc[:300]

## Q1. Getting the embeddings model

In [85]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)


In [86]:
answer_llm = df.iloc[0].answer_llm
embeddings_answer_llm = embedding_model.encode(answer_llm)


In [87]:
list(embeddings_answer_llm)[0]

-0.42244655

## Q2. Computing the dot product

In [88]:
df["answer_llm_embeddings"] = df["answer_llm"].apply(embedding_model.encode)

In [89]:
df["answer_orig_embeddings"] = df["answer_orig"].apply(embedding_model.encode)

In [90]:
df.head()

,answer_llm,answer_orig,document,question,course,answer_llm_embeddings,answer_orig_embeddings
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244655, -0.22485591, -0.32405853, -0.284...","[-0.030214049, -0.3444384, -0.2807624, 0.06150..."
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068187, 0.047848765, -0.31510985, -0.210...","[-0.030214049, -0.3444384, -0.2807624, 0.06150..."
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.058813807, -0.33736983, -0.3615757, 0.0217...","[-0.030214049, -0.3444384, -0.2807624, 0.06150..."
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753698, -0.008133725, -0.2171992, -0.110...","[-0.030214049, -0.3444384, -0.2807624, 0.06150..."
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969395, -0.50050926, -0.16598433, 0.3066...","[-0.030214049, -0.3444384, -0.2807624, 0.06150..."


In [91]:
df["scores"] = df.apply(lambda row: row["answer_llm_embeddings"].dot(row["answer_orig_embeddings"]), axis=1)

In [92]:
df.scores.describe()

count    300.000000
mean      27.495996
std        6.384744
min        4.547924
25%       24.307847
50%       28.336863
75%       31.674307
max       39.476013
Name: scores, dtype: float64

In [93]:
df.scores.quantile(0.75)

31.674307346343994

## Q3. Computing the cosine

In [94]:
def norm_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [95]:
df["answer_llm_emb_norm"] = df["answer_llm_embeddings"].apply(norm_vector)
df["answer_orig_emb_norm"] = df["answer_orig_embeddings"].apply(norm_vector)

In [96]:
df["scores"] = df.apply(lambda row: row["answer_llm_emb_norm"].dot(row["answer_orig_emb_norm"]), axis=1)

In [97]:
df.scores.quantile(0.75)

0.8362346887588501

## Q4. Rouge

In [98]:
r = df.iloc[10]
assert r.document == '5170565b'

In [99]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores['rouge-1']['f']

0.45454544954545456

## Q5. Average rouge score

In [100]:
f_scores = [scores[rouge_score_name]['f'] for rouge_score_name in scores]

In [101]:
np.mean(f_scores)

0.35490034990035496

## Q6. Average rouge score for all the data points

In [102]:
def get_rouge_scores(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return {rouge_score_name: scores[rouge_score_name]['f'] for rouge_score_name in scores}

df["rouge"] = df.apply(lambda row: get_rouge_scores(row), axis=1)

In [103]:
rouge_df = pd.DataFrame(list(df.rouge.values))
rouge_df.head()

,rouge-1,rouge-2,rouge-l
0,0.095238,0.028169,0.095238
1,0.125000,0.055556,0.093750
2,0.415584,0.177778,0.389610
3,0.216216,0.047059,0.189189
4,0.142076,0.033898,0.120219


In [104]:
rouge_df["rouge-2"].mean()

0.20696501983423318